In [ ]:
# Import von benötigten Modulen
import ipywidgets as widgets
from IPython.display import  clear_output
import random
import string
import warnings
warnings.filterwarnings('ignore')

# Gundvariablen
NUM_LETTERS = 5
NUM_GUESSES = 6
WORDS_PATH = "wordlist.txt"
WORDS_PATH_GUESS = "wordlist v2.txt"

# Benötigte Widgets
text_input = widgets.Text(description='Dein Wort:')
submit_button = widgets.Button(description='Überprüfen')
output_area = widgets.Output()
canvas_output = widgets.Output()
alphabet_output = widgets.Output()
restart_button = widgets.Button(description="Neu starten")
debug = widgets.Output()

# Funktionen um zu ratendes Wort zu prüfen und auszuwählen
def get_random_word(word_list):
    words = [
        word.upper() for word in word_list 
        if len(word) == NUM_LETTERS and all(letter in string.ascii_letters for letter in word)
    ]
    return random.choice(words) if words else None

def read_word_list():
    with open(WORDS_PATH, "r") as file:
        return [line.strip().upper() for line in file.read().splitlines()]

def read_word_list_guess():
    with open(WORDS_PATH_GUESS, "r") as file:
        return [line.strip().upper() for line in file.read().splitlines()]

# Festlegung des zu ratenden Wortes und des aktuellen Ratversuches
word_list = read_word_list()
word_list_guess = read_word_list_guess()
word = get_random_word(word_list)
guesses = ["_" * NUM_LETTERS for _ in range(NUM_GUESSES)]
current_guess = 0

# Überprüfung des Rateversuches
def check_guess(b):
    global current_guess
    with output_area:
        clear_output(wait=True)
        
        guess = text_input.value.upper()
        if len(guess) != NUM_LETTERS or not guess.isalpha():
            print("Ungültige Eingabe. Versuche es erneut.")
            return     
        
        if guess not in word_list_guess:
            print("Das Wort existiert nicht. Versuche es erneut.")
            return

        # Rateversuch zählt nur, wenn das Wort in der Liste ist
        current_guess += 1
        guesses[current_guess - 1] = guess  # Aktualisiere den aktuellen Rateversuch
        update_canvas(guesses, word)
        update_alphabet(guesses)  # Aktualisiere das Alphabet

        if guess == word:
            print("Richtig! Du hast das Wort erraten.")
            submit_button.disabled = True  # Deaktiviere den Button, wenn das Spiel vorbei ist
        
        elif current_guess >= NUM_GUESSES:
            # Zeige die Meldung und das zu ratende Wort, wenn 6 mal falsch geraten wurde
            print(f"Spiel vorbei. Das Wort war: {word}")
            submit_button.disabled = True  # Deaktiviere den Button, da das Spiel vorbei ist
        
        else:
            # Feedback für den Spieler, wenn das Spiel noch nicht vorbei ist
            print(f"Versuch {current_guess} von {NUM_GUESSES}. Versuche es erneut.")
    
        text_input.value = ''  # Eingabefeld leeren nach jeder Überprüfung
        
# Darstellung der richtigen / teils richtigen Buchstaben
def update_canvas(guesses, word):
    with canvas_output:
        clear_output(wait=True)
        display(f"Versuche: {current_guess}/{NUM_GUESSES}")
        for guess in guesses:
            styled_guess = ""
            #Überprüfung für die Anzahl der Vorkommen eines Buchstabens
            word_occurrences = {letter: word.count(letter) for letter in set(word)}
            guess_occurrences_correct = {letter: 0 for letter in set(word)}
            
            # Zuerst die korrekten Positionen markieren und die Vorkommen zählen
            for i, letter in enumerate(guess):
                if letter == word[i]:
                    guess_occurrences_correct[letter] += 1
            
            for i, letter in enumerate(guess):
                if letter == word[i]:
                    # Richtig und an der richtigen Stelle, fett und grün hinterlegt
                    styled_guess += f"<span style='background-color: lightgreen; font-weight: bold;'>{letter}</span> "
                elif letter in word:
                    # Überprüfen, ob die Anzahl der korrekten Vorkommen kleiner ist als die im Wort und nur dann orange markieren
                    if guess.count(letter) <= word_occurrences[letter] or guess_occurrences_correct[letter] < word_occurrences[letter]:
                        styled_guess += f"<span style='background-color: orange; font-weight: bold;'>{letter}</span> "
                        if guess[i] != word[i]:
                            guess_occurrences_correct[letter] += 1
                    else:
                        styled_guess += f"<span style='font-weight: bold;'>{letter}</span> "
                else:
                    # Falsch, nur fett
                    styled_guess += f"<span style='font-weight: bold;'>{letter}</span> "
            display(widgets.HTML(value=styled_guess))

# Alphabet aktuallisieren
def update_alphabet(guesses):
    used_letters = set(''.join(guesses))
    alphabet_display = ""
    extended_alphabet = string.ascii_uppercase + "ÄÖÜ"  # Erweitert das Alphabet um Umlaute

    for letter in extended_alphabet:
        if letter in used_letters:
            # Markiere benutzte Buchstaben mit einer anderen Farbe (hier: Rot)
            alphabet_display += f"<span style='color: red; font-weight: bold'>{letter}</span> "
        else:
            alphabet_display += f"<span style='font-weight: bold;'>{letter}</span> "
    with alphabet_output:
        clear_output(wait=True)
        display(widgets.HTML(value=alphabet_display))

# Neustart-Funktion
def restart_game(b=None):
    global word, guesses, current_guess, submit_button
    word = get_random_word(word_list)
    guesses = ["_" * NUM_LETTERS for _ in range(NUM_GUESSES)]
    current_guess = 0
    submit_button.disabled = False  # Aktiviert den Überprüfen-Button wieder
    with output_area:
        clear_output(wait=False)
    with canvas_output:
        clear_output(wait=True)
    text_input.value = ''
    update_canvas(guesses, word)
    update_alphabet(guesses)  # Initialisiere das Alphabet

# Anzeigen der UI-Elemente
display(text_input, submit_button, output_area, canvas_output, alphabet_output, restart_button, debug)

# Spiel bei Initialisierung starten
restart_game()

# Buttons verknüpfen
restart_button.on_click(restart_game)         
submit_button.on_click(check_guess)
text_input.on_submit(check_guess)